1. Introduction
===================

In this mission, we'll analyze geographic data on brazilian municipalities. The dataset comes from [IBGE - Instituto Brasileiro de Geografia e Estatística](https://downloads.ibge.gov.br/downloads_estatisticas.htm). The goal is to expand our skills about visualization on [choropleth maps](https://en.wikipedia.org/wiki/Choropleth_map). 

A choropleth map is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income. 



In [1]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random

In [2]:
# dataset name
dataset_pop_2017 = os.path.join('data', 'population_2017.csv')

# read the data to a dataframe
data2017 = pd.read_csv(dataset_pop_2017)

In [3]:
# see the first five rows
data2017.head()

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA
0,RO,11.0,15.0,Alta Floresta D'Oeste,25437.0
1,RO,11.0,23.0,Ariquemes,107345.0
2,RO,11.0,31.0,Cabixi,6224.0
3,RO,11.0,49.0,Cacoal,88507.0
4,RO,11.0,56.0,Cerejeiras,17934.0


In [4]:
# use more appropriate names for columns
data2017.columns = [['State', 'Id_State', 'Id_City',
                    'City', 'Population']]

data2017.head()

,State,Id_State,Id_City,City,Population
0,RO,11.0,15.0,Alta Floresta D'Oeste,25437.0
1,RO,11.0,23.0,Ariquemes,107345.0
2,RO,11.0,31.0,Cabixi,6224.0
3,RO,11.0,49.0,Cacoal,88507.0
4,RO,11.0,56.0,Cerejeiras,17934.0


In [5]:
# filtering data about RN state
dataRN = data2017[data2017['State'] == 'RN']

# sort dataset by city name
dataRN = dataRN.sort_values('Population',ascending=False)

dataRN.head()


,State,Id_State,Id_City,City,Population
1163,RN,24.0,8102.0,Natal,885180.0
1162,RN,24.0,8003.0,Mossoró,295619.0
1110,RN,24.0,3251.0,Parnamirim,254709.0
1205,RN,24.0,12005.0,São Gonçalo do Amarante,101492.0
1152,RN,24.0,7104.0,Macaíba,80031.0


In [6]:
dataRN.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 1163 to 1240
Data columns (total 5 columns):
State         167 non-null object
Id_State      167 non-null float64
Id_City       167 non-null float64
City          167 non-null object
Population    167 non-null float64
dtypes: float64(3), object(2)
memory usage: 7.8+ KB



2. Geodata BR - Brazil
===================

This project contains files [Geojson](http://geojson.org/) with the perimeters
of Brazilian municipalities group by state.

source: https://github.com/tbrugz/geodata-br



### North Region
* AC / Acre - [geojson/geojs-12-mun.json](geojson/geojs-12-mun.json)
* AM / Amazonas - [geojson/geojs-13-mun.json](geojson/geojs-13-mun.json)
* AP / Amapá - [geojson/geojs-16-mun.json](geojson/geojs-16-mun.json)
* PA / Pará  - [geojson/geojs-15-mun.json](geojson/geojs-15-mun.json)
* RO / Rondônia - [geojson/geojs-11-mun.json](geojson/geojs-11-mun.json)
* RR / Roraima - [geojson/geojs-14-mun.json](geojson/geojs-14-mun.json)
* TO / Tocantins - [geojson/geojs-17-mun.json](geojson/geojs-17-mun.json)


### Northeast Region
* AL / Alagoas - [geojson/geojs-27-mun.json](geojson/geojs-27-mun.json)
* BA / Bahia - [geojson/geojs-29-mun.json](geojson/geojs-29-mun.json)
* CE / Ceará - [geojson/geojs-23-mun.json](geojson/geojs-23-mun.json)
* MA / Maranhão - [geojson/geojs-21-mun.json](geojson/geojs-21-mun.json)
* PB / Paraíba - [geojson/geojs-25-mun.json](geojson/geojs-25-mun.json)
* PE / Pernambuco - [geojson/geojs-26-mun.json](geojson/geojs-26-mun.json)
* PI / Piauí - [geojson/geojs-22-mun.json](geojson/geojs-22-mun.json)
* RN / Rio Grande do Norte - [geojson/geojs-24-mun.json](geojson/geojs-24-mun.json)
* SE / Sergipe - [geojson/geojs-28-mun.json](geojson/geojs-28-mun.json)


### Southeast Region
* ES / Espíriro Santo - [geojson/geojs-32-mun.json](geojson/geojs-32-mun.json)
* MG / Minas Gerais - [geojson/geojs-31-mun.json](geojson/geojs-31-mun.json)
* RJ / Rio de Janeiro - [geojson/geojs-33-mun.json](geojson/geojs-33-mun.json)
* SP / São Paulo - [geojson/geojs-35-mun.json](geojson/geojs-35-mun.json)


### South Region
* PR / Paraná - [geojson/geojs-41-mun.json](geojson/geojs-41-mun.json)
* RS / Rio Grande do Sul - [geojson/geojs-43-mun.json](geojson/geojs-43-mun.json)
* SC / Santa Catarina - [geojson/geojs-42-mun.json](geojson/geojs-42-mun.json)


### Central-west Region
* DF / Distrito Federal - [geojson/geojs-53-mun.json](geojson/geojs-53-mun.json) 
* GO / Goiás - [geojson/geojs-52-mun.json](geojson/geojs-52-mun.json)
* MT / Mato Grosso - [geojson/geojs-51-mun.json](geojson/geojs-51-mun.json)
* MS / Mato Grosso do Sul - [geojson/geojs-50-mun.json](geojson/geojs-50-mun.json)


### Brazil
* BR / Brazil - [geojson/geojs-100-mun.json](geojson/geojs-100-mun.json)




3. Customization 
===================

If you need, it is possible generate and customize GeoJson files from online editors. The project [GeoJson.io](http://geojson.io/) is an excelent path to give the first steps. 



4. Importing GeoJson files
===================

In [7]:
# searching the files in geojson/geojs-xx-mun.json
br_states = os.path.join('geojson', 'geojs-24-mun.json')

# load the data and use 'latin-1'encoding because the accent
geo_json_data = json.load(open(br_states,encoding='latin-1'))


In [8]:
geo_json_data

{'features': [{'geometry': {'coordinates': [[[-36.6752824479, -6.2695704427],
      [-36.6721661976, -6.2748710057],
      [-36.6621971359, -6.2781206182],
      [-36.6544080838, -6.2718175581],
      [-36.6302770363, -6.2681148661],
      [-36.625658466, -6.2854823428],
      [-36.6151351174, -6.292907263],
      [-36.6042576558, -6.2899699655],
      [-36.595723618, -6.2878348922],
      [-36.592155387, -6.2965905174],
      [-36.5839051708, -6.300143773],
      [-36.5844180215, -6.3059773274],
      [-36.5641014276, -6.3124566042],
      [-36.5596074435, -6.3270377373],
      [-36.5544893832, -6.3304039133],
      [-36.5528973655, -6.3394789997],
      [-36.5476492661, -6.3439752057],
      [-36.5505221875, -6.3536185214],
      [-36.5505219847, -6.3614560301],
      [-36.545503695, -6.3634059823],
      [-36.5420902739, -6.3725037009],
      [-36.517493878, -6.3778133912],
      [-36.5040431445, -6.3860516598],
      [-36.5071574456, -6.3997173571],
      [-36.5306368773, -6.451671

In [9]:
# http://cidades.ibge.gov.br/painel/historico.php?codmun=241030
# Presidente Juscelino city changes your name to Serra Caiada
geo_json_data['features'][112]['properties']['description'] = 'Serra Caiada'
geo_json_data['features'][112]['properties']['name'] = 'Serra Caiada'


In [10]:
cities = []
# list all cities in the state
for city in geo_json_data['features']:
        cities.append(city['properties']['description'])
cities


['Acari',
 'Açu',
 'Afonso Bezerra',
 'Água Nova',
 'Alexandria',
 'Almino Afonso',
 'Alto do Rodrigues',
 'Angicos',
 'Antônio Martins',
 'Apodi',
 'Areia Branca',
 'Arês',
 'Augusto Severo',
 'Baía Formosa',
 'Baraúna',
 'Barcelona',
 'Bento Fernandes',
 'Bodó',
 'Bom Jesus',
 'Brejinho',
 'Caiçara do Norte',
 'Caiçara do Rio do Vento',
 'Caicó',
 'Campo Redondo',
 'Canguaretama',
 'Caraúbas',
 'Carnaúba dos Dantas',
 'Carnaubais',
 'Ceará-Mirim',
 'Cerro Corá',
 'Coronel Ezequiel',
 'Coronel João Pessoa',
 'Cruzeta',
 'Currais Novos',
 'Doutor Severiano',
 'Parnamirim',
 'Encanto',
 'Equador',
 'Espírito Santo',
 'Extremoz',
 'Felipe Guerra',
 'Fernando Pedroza',
 'Florânia',
 'Francisco Dantas',
 'Frutuoso Gomes',
 'Galinhos',
 'Goianinha',
 'Governador Dix-Sept Rosado',
 'Grossos',
 'Guamaré',
 'Ielmo Marinho',
 'Ipanguaçu',
 'Ipueira',
 'Itajá',
 'Itaú',
 'Jaçanã',
 'Jandaíra',
 'Janduís',
 'Januário Cicco',
 'Japi',
 'Jardim de Angicos',
 'Jardim de Piranhas',
 'Jardim do Seridó

In [11]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(geo_json_data).add_to(m)

m

5. Drawing the Choropleth
======================

In [12]:
# verify again the datarn
dataRN.head()

,State,Id_State,Id_City,City,Population
1163,RN,24.0,8102.0,Natal,885180.0
1162,RN,24.0,8003.0,Mossoró,295619.0
1110,RN,24.0,3251.0,Parnamirim,254709.0
1205,RN,24.0,12005.0,São Gonçalo do Amarante,101492.0
1152,RN,24.0,7104.0,Macaíba,80031.0


Then, in playing with keyword arguments, you can get a choropleth in (almost) one line :


In [13]:
# Create a map object
map_rn = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

In [14]:
# create a threshold of legend
threshold_scale = np.linspace(dataRN['Population'].min(),
                              dataRN['Population'].max()/4, 6, dtype=int).tolist()

In [15]:
map_rn.choropleth(
    geo_data=geo_json_data,
    data=dataRN,
    columns=['City', 'Population'],
    key_on='feature.properties.description',
    fill_color='YlOrRd',
    legend_name='Population estimation (2017)',
    name ='Choropleth Map for population estimation of RN',
    highlight=True,
    threshold_scale = threshold_scale
)

folium.LayerControl().add_to(map_rn)

map_rn

In [16]:
# search all features
for feature in geo_json_data['features']:
    # get the city name
    city_name = feature['properties']['name']
    
    # filter by the most populated cities
    if city_name in dataRN.City[:10].values:
        
        # take the coordinates (lat,log) of city
        geom = feature['geometry']['coordinates']
    
        # create a polygon using all coordinates
        polygon = Polygon(geom[0])
    
        # calculate the centroid of polygon
        lng, lat = polygon.centroid.coords.xy
    
        # Draw a small circle
        folium.CircleMarker([lat[0],lng[0]], radius=2, 
                        popup='%s Population: %d' % \
                            (city_name, dataRN[dataRN.City==city_name]['Population']),
                        color='red').add_to(map_rn)

In [17]:
map_rn

6 Importing Geojson files from overpass-turbo project
==

Source: http://overpass-turbo.eu/

Query to [Natal neighborhoods](http://wiki.openstreetmap.org/wiki/Natal#Bairros):
>```python
[out:json][timeout:25];
{{geocodeArea:Natal RN Brasil}}->.searchArea;
(
  relation["admin_level"="10"](area.searchArea);
);
out body;
>>;
out skel qt;
```

In [18]:
# import geojson file about natal neighborhood
natal_neigh = os.path.join('geojson', 'natal.geojson')

# load the data and use 'UTF-8'encoding
geo_json_natal = json.load(open(natal_neigh,encoding='UTF-8'))


In [19]:
geo_json_natal

{'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.',
 'features': [{'geometry': {'coordinates': [[[-35.2251535, -5.8800875],
      [-35.2245789, -5.8789859],
      [-35.2235407, -5.8773961],
      [-35.2216713, -5.8748329],
      [-35.219967, -5.8725269],
      [-35.219495, -5.8717499],
      [-35.2183771, -5.8693635],
      [-35.2158321, -5.8640165],
      [-35.2159318, -5.8639635],
      [-35.2160512, -5.8638541],
      [-35.2207751, -5.8610102],
      [-35.226799, -5.857474],
      [-35.2287216, -5.8563299],
      [-35.2288872, -5.8562443],
      [-35.2292113, -5.8560767],
      [-35.2293013, -5.8560301],
      [-35.2316996, -5.854504],
      [-35.2330707, -5.8537719],
      [-35.2333137, -5.8512412],
      [-35.2351934, -5.85008],
      [-35.2363521, -5.8486285],
      [-35.2382532, -5.846884],
      [-35.2386823, -5.8464656],
      [-35.2394376, -5.8460131],
      [-35.240459, -5.8452702],
      [-35.2409139, -5.

In [20]:
neighborhood = []
# list all neighborhoods
for neigh in geo_json_natal['features']:
        neighborhood.append(neigh['properties']['name'])
neighborhood


['Pitimbu',
 'Planalto',
 'Ponta Negra',
 'Neópolis',
 'Capim Macio',
 'Lagoa Azul',
 'Pajuçara',
 'Lagoa Seca',
 'Barro Vermelho',
 'Candelária',
 'Praia do Meio',
 'Rocas',
 'Santos Reis',
 'Redinha',
 'Salinas',
 'Igapó',
 'Nossa Senhora da Apresentação',
 'Potengi',
 'Ribeira',
 'Cidade Alta',
 'Alecrim',
 'Nordeste',
 'Quintas',
 'Bom Pastor',
 'Dix-Sept Rosado',
 'Nossa Senhora de Nazaré',
 'Lagoa Nova',
 'Mãe Luiza',
 'Nova Descoberta',
 'Tirol',
 'Petrópolis',
 'Areia Preta',
 'Cidade Nova',
 'Cidade da Esperança',
 'Felipe Camarão',
 'Guarapes']

In [21]:
len(neighborhood)

36

In [22]:
# return a number of points inside the polygon
def generate_random(number, polygon, neighborhood):
    list_of_points = []
    minx, miny, maxx, maxy = polygon.bounds
    counter = 0
    while counter < number:
        x = random.uniform(minx, maxx)
        y = random.uniform(miny, maxy)
        pnt = Point(x, y)
        if polygon.contains(pnt):
            list_of_points.append([x,y,neighborhood])
            counter += 1
    return list_of_points

In [23]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=11,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(geo_json_natal).add_to(m)

In [24]:
number_of_points = 3

# search all features
for feature in geo_json_natal['features']:
    # get the name of neighborhood
    neighborhood = feature['properties']['name']
    # take the coordinates (lat,log) of neighborhood
    geom = feature['geometry']['coordinates']
    # create a polygon using all coordinates
    polygon = Polygon(geom[0])
    # return number_of_points by neighborhood as a list [[log,lat],....]
    points = generate_random(number_of_points,polygon, neighborhood)
    # iterate over all points and print in the map
    for i,value in enumerate(points):
        log, lat, name = value 
        # Draw a small circle
        folium.CircleMarker([lat,log],
                    radius=2,
                    popup='%s %s%d' % (name, '#', i),
                   color='red').add_to(m)
m

7. Uber API
==

1. create user in https://developer.uber.com/
2. create an app in https://developer.uber.com/
3. install uber-rides package
>```python
!pip install uber-rides
```


In [25]:
!pip install uber-rides

### 7.1 Create an Uber session with a server token

Once you’ve created your app, you’ll be given a **server_token**, **client_id**, & **client_secret**. These are used to authenticate your application and the rider when calling the API.

In [26]:
from uber_rides.session import Session
from uber_rides.client import UberRidesClient

session = Session(server_token='fpvLpMxMUxI4sRMKgJ_JMAahiLfg1D06ngYuJAPa')
client = UberRidesClient(session)

###  7.2 Get a list of available products

The [Products endpoint](https://developer.uber.com/docs/riders/references/api/v1.2/products-get) returns information about the Uber products offered at a given location. The response includes the display name and other details about each product, and lists the products in the proper display order.

In some markets, the list of products returned from this endpoint may vary by the time of day due to time restrictions on when that product may be utilized.


In [27]:
response = client.get_products(-5.8323,-35.2054)

# API - get/products
products = response.json.get('products')

In [28]:
print(products)

[{'upfront_fare_enabled': True, 'capacity': 4, 'product_id': '65cb1829-9761-40f8-acc6-92d700fe2924', 'price_details': {'service_fees': [{'fee': 0.75, 'name': 'Booking fee'}], 'cost_per_minute': 0.17, 'distance_unit': 'km', 'minimum': 6.75, 'cost_per_distance': 1.2, 'base': 2.5, 'cancellation_fee': 6.75, 'currency_code': 'BRL'}, 'image': 'http://d1a3f4spazzrp4.cloudfront.net/car-types/mono/mono-uberx.png', 'cash_enabled': False, 'shared': False, 'short_description': 'uberX', 'display_name': 'uberX', 'product_group': 'uberx', 'description': 'THE LOW-COST UBER'}, {'upfront_fare_enabled': True, 'capacity': 4, 'product_id': 'bf8f99ca-f5f2-40d4-8ffc-52f1e2b17138', 'price_details': {'service_fees': [{'fee': 0.75, 'name': 'Booking fee'}], 'cost_per_minute': 0.2, 'distance_unit': 'km', 'minimum': 7.75, 'cost_per_distance': 1.44, 'base': 3.0, 'cancellation_fee': 6.0, 'currency_code': 'BRL'}, 'image': 'http://d1a3f4spazzrp4.cloudfront.net/car-types/mono/mono-uberx.png', 'cash_enabled': False, 'sh

In [29]:
products[0]

{'capacity': 4,
 'cash_enabled': False,
 'description': 'THE LOW-COST UBER',
 'display_name': 'uberX',
 'image': 'http://d1a3f4spazzrp4.cloudfront.net/car-types/mono/mono-uberx.png',
 'price_details': {'base': 2.5,
  'cancellation_fee': 6.75,
  'cost_per_distance': 1.2,
  'cost_per_minute': 0.17,
  'currency_code': 'BRL',
  'distance_unit': 'km',
  'minimum': 6.75,
  'service_fees': [{'fee': 0.75, 'name': 'Booking fee'}]},
 'product_group': 'uberx',
 'product_id': '65cb1829-9761-40f8-acc6-92d700fe2924',
 'shared': False,
 'short_description': 'uberX',
 'upfront_fare_enabled': True}

In [30]:
products[1]

{'capacity': 4,
 'cash_enabled': False,
 'description': 'THE LOW-COST UBER',
 'display_name': 'UberSELECT',
 'image': 'http://d1a3f4spazzrp4.cloudfront.net/car-types/mono/mono-uberx.png',
 'price_details': {'base': 3.0,
  'cancellation_fee': 6.0,
  'cost_per_distance': 1.44,
  'cost_per_minute': 0.2,
  'currency_code': 'BRL',
  'distance_unit': 'km',
  'minimum': 7.75,
  'service_fees': [{'fee': 0.75, 'name': 'Booking fee'}]},
 'product_group': 'uberx',
 'product_id': 'bf8f99ca-f5f2-40d4-8ffc-52f1e2b17138',
 'shared': False,
 'short_description': 'SELECT',
 'upfront_fare_enabled': True}

### 7.3 Get price and time estimates

The [Time Estimates endpoint](https://developer.uber.com/docs/riders/references/api/v1.2/estimates-time-get) returns ETAs (estimate time of arrivals) for all products currently available at a given location, with the ETA for each product expressed as integers in seconds. If a product returned from [GET /v1.2/products](https://developer.uber.com/docs/riders/references/api/v1.2/products-get) is not returned from this endpoint for a given latitude/longitude pair then there are currently none of that product available to request. We recommend that this endpoint be called every minute to provide the most accurate, up-to-date ETAs.

The [Price Estimates endpoint](https://developer.uber.com/docs/riders/references/api/v1.2/estimates-price-get) returns an estimated price range for each product offered at a given location. The price estimate is provided as a formatted string with the full price range and the localized currency symbol.

The response also includes low and high estimates, and the [ISO 4217](http://en.wikipedia.org/wiki/ISO_4217) currency code for situations requiring currency conversion. When surge is active for a particular product, its **surge_multiplier** will be greater than **1**, but the price estimate already factors in this multiplier.



In [31]:
response = client.get_price_estimates(
    start_latitude=-5.8323,
    start_longitude=-35.2054,
    end_latitude= -5.8734,
    end_longitude=-35.1776,
    seat_count=2
)

In [32]:
type(response)

uber_rides.request.Response

In [33]:
response.json

{'prices': [{'currency_code': 'BRL',
   'display_name': 'uberX',
   'distance': 5.02,
   'duration': 840,
   'estimate': 'R$13-17',
   'high_estimate': 17.0,
   'localized_display_name': 'uberX',
   'low_estimate': 13.0,
   'product_id': '65cb1829-9761-40f8-acc6-92d700fe2924'},
  {'currency_code': 'BRL',
   'display_name': 'UberSELECT',
   'distance': 5.02,
   'duration': 840,
   'estimate': 'R$16-20',
   'high_estimate': 20.0,
   'localized_display_name': 'UberSELECT',
   'low_estimate': 16.0,
   'product_id': 'bf8f99ca-f5f2-40d4-8ffc-52f1e2b17138'}]}

In [34]:
response.json.get('prices')

[{'currency_code': 'BRL',
  'display_name': 'uberX',
  'distance': 5.02,
  'duration': 840,
  'estimate': 'R$13-17',
  'high_estimate': 17.0,
  'localized_display_name': 'uberX',
  'low_estimate': 13.0,
  'product_id': '65cb1829-9761-40f8-acc6-92d700fe2924'},
 {'currency_code': 'BRL',
  'display_name': 'UberSELECT',
  'distance': 5.02,
  'duration': 840,
  'estimate': 'R$16-20',
  'high_estimate': 20.0,
  'localized_display_name': 'UberSELECT',
  'low_estimate': 16.0,
  'product_id': 'bf8f99ca-f5f2-40d4-8ffc-52f1e2b17138'}]

In [35]:
wait_time = client.get_pickup_time_estimates(-5.8323,
                                 -35.2054,
                                'bf8f99ca-f5f2-40d4-8ffc-52f1e2b17138')

In [36]:
wait_time.json

{'times': [{'display_name': 'UberSELECT',
   'estimate': 360,
   'localized_display_name': 'UberSELECT',
   'product_id': 'bf8f99ca-f5f2-40d4-8ffc-52f1e2b17138'}]}

In [37]:
wait_time.json.get('times')[0]['estimate']

360

### 7.4 Rating limits

The Uber API enforces rate limits to help distribute resources among apps. Based on registered app’s **server_token**, there are a limit to 2000 requests per hour.


<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

1. Motivation: [Uber seems to offer better service in areas with more white people](https://www.washingtonpost.com/news/wonk/wp/2016/03/10/uber-seems-to-offer-better-service-in-areas-with-more-white-people-that-raises-some-tough-questions/?utm_term=.b12202a9ed14)
2. Interval: 3min; duration: 2 week
    - query data about X uniform points for each neighborhood in Natal-RN.
    - X must be selected according to rating limits of Uber API.  
    - compute the ETA average for all neighborhoods.
    - use a choropleth map to generate an aesthetic plot.

<br>
<div class="alert alert-success">
<b>Answer.</b>
</div>

Source: https://github.com/ycaroravel/Analyzing-Uber-Rides/

### Read the dataframe

In [38]:
# dataset name (~16MB)
uber_path = os.path.join('data', 'uber-eta.csv')

# read the data to a dataframe
uber_eta_df = pd.read_csv(uber_path)

### Exploratory data analysis

In [39]:
# collection period was 23 days
print('first datatime: ', uber_eta_df.loc[0,'datetime'])
print('last datatime: ',  uber_eta_df[-1:]['datetime'])

first datatime:  2017-10-18 23:00:04.989364
last datatime:  179319    2017-10-31 22:21:46.261904
Name: datetime, dtype: object


In [40]:
uber_eta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179320 entries, 0 to 179319
Data columns (total 6 columns):
datetime      179320 non-null object
log           179320 non-null float64
lat           179320 non-null float64
neigh         179320 non-null object
point_hash    179320 non-null object
eta           179320 non-null float64
dtypes: float64(3), object(3)
memory usage: 8.2+ MB


### Analysis

In [41]:
uber_analysis = uber_eta_df.pivot_table(index='neigh', values='eta', aggfunc='mean')

In [42]:
uber_analysis.head()

,eta
neigh,
Alecrim,237.410678
Areia Preta,256.493976
Barro Vermelho,261.372667
Bom Pastor,281.253012
Candelária,295.979133


In [43]:
uber_analysis = uber_analysis.reset_index()
uber_analysis.head()

,neigh,eta
0,Alecrim,237.410678
1,Areia Preta,256.493976
2,Barro Vermelho,261.372667
3,Bom Pastor,281.253012
4,Candelária,295.979133


### Create the map

In [44]:
# Create a map object
uber_map = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=11,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(geo_json_natal).add_to(uber_map)

In [45]:
# create a threshold of legend
threshold_scale = np.linspace(uber_analysis.eta.min(),
                              uber_analysis.eta.max(), 6, dtype=int).tolist()

In [46]:
uber_map.choropleth(
    geo_data=geo_json_natal,
    data=uber_analysis,
    columns=['neigh','eta'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    legend_name='Uber ETA (avg.) during 23 days (18 oct to 31 oct)',
    name ='Choropleth Map Uber ETA (avg.)',
    highlight=True,
    threshold_scale = threshold_scale
)

folium.LayerControl().add_to(uber_map)

uber_map

In [47]:
# search all features
for feature in geo_json_natal['features']:
    
    # get the name of neighborhood
    neigh = feature['properties']['name']
    
    # take the coordinates (lat,log) of neighborhood
    geom = feature['geometry']['coordinates']
    
    # create a polygon using all coordinates
    polygon = Polygon(geom[0])
    
    # calculate the controid of neighborhood
    x,y = polygon.centroid.coords.xy
    lat = y[0]
    log = x[0]
    
    folium.CircleMarker([lat,log],
                        radius=6,
                        popup='%s, ETA (avg.): %d' %\
                        (neigh,
                         uber_analysis[uber_analysis.neigh == neigh]['eta']),
                        color='red').add_to(uber_map)

In [48]:
uber_map